In [1]:
# -*- encoding: utf-8 -*-
import sys
import logging
import fasttext

In [2]:
# reload(sys)
# sys.setdefaultencoding('utf-8')

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [4]:
def trainSkipgram(inFile,mdlfile):
    # Skipgram model
    model = fasttext.skipgram(input_file=inFile,\
                              output=mdlfile,\
                              lr=0.1,\
                              dim=200,\
                              epoch=10,\
                              word_ngrams=3,\
                              bucket=5000000)
#     print "Skipgram model words:",model.words # list of words in dictionary

In [5]:
def trainCBOW(inFile,mdlfile):
    # CBOW model
    model = fasttext.cbow(input_file=inFile,\
                              output=mdlfile,\
                              lr=0.1,\
                              dim=200,\
                              epoch=10,\
                              word_ngrams=3,\
                              bucket=5000000)
#     print "CBOW model words:",model.words # list of words in dictionary

In [6]:
def trainCLF(inFile,model,vec):
    #训练模型
    classifier = fasttext.supervised(input_file=inFile,\
                                     output=model,\
                                     label_prefix="__label__",\
                                     dim = 200,\
                                     word_ngrams = 2,\
                                     epoch = 10,\
                                     pretrained_vectors = vec)

In [7]:
def test(inFile,model):
    #load训练好的模型
    classifier = fasttext.load_model(model)
    
    result = classifier.test(inFile)
    print 'P@1:', result.precision
    print 'R@1:', result.recall
    print 'Number of examples:', result.nexamples

In [8]:
# # 获取测试数据
def getTestSet(inFile):
    # 训练集
    train_set=[]
    # 情感标签集
    docid_set = []
   
    # 读入训练数据
    f=open(inFile)
    lines=f.readlines()
    for line in lines:
        article = line.replace('\n','').split(" ")
        
        # 文章id
        docid_set.append(article[0])
       
        # 内容
        content = article[1:]
        train_set.append(content)        

    f.close()
    return (train_set,docid_set)

In [9]:
# 把分词以后的结果写入文件
def writeFile(outputfile,newline):
    
    fw = open(outputfile, 'ab')
    fw.write(newline.encode("utf-8"))
    fw.close()

In [10]:
def pred(inFile,model,outputfile):
    
    # 读入测试文件
    testData,docid = getTestSet(inFile)
    for t in testData:
        for i in t:
            print i
   
    #load训练好的模型
    classifier = fasttext.load_model(model,label_prefix='__label__')
    for idx,text in enumerate(testData):
#         print text
        label = classifier.predict(text)
#         print len(label)
        newline = docid[idx] + " " + label +  "\n"
        
        writeFile(outputfile,newline)

In [11]:
def loadMdl(mdl):
    model = fasttext.load_model(mdl, encoding='utf-8')

In [ ]:
def main():
   
    # 定义文件路径
    dataPath = "/home/hadoop/DataSencise/bdci2017/BDCI2017-360/data/"
    mdlPath = "/home/hadoop/DataSencise/bdci2017/BDCI2017-360/model/"
    
    # 定义训练文件名
#     inFile = dataPath + "train/train_output_title.tsv"
    inFile = dataPath + "train/train_all.tsv"
    
    # test 文件
    testFile = dataPath + "test/test_all.tsv"
    
    # test 文件
    outFile = dataPath + "test/pred_output.tsv"
    
    # 定义vec模型
    model_skipgram = mdlPath + "vec_skipgram"
    # 定义vec模型
    model_CBOW = mdlPath + "vec_CBOW"
    
    # 定义clf模型
    model_clf_sg = mdlPath + "clf_model_sg"
    
    # 定义clf模型
    model_clf_CBOW = mdlPath + "clf_model_CBOW"

    # 训练vector
    trainSkipgram(inFile,model_skipgram)

    # 训练vector
#     trainCBOW(inFile,model_CBOW)
    
    vec_skipgram = model_skipgram + ".vec"
    vec_CBOW = model_CBOW + ".vec"
    
    # 分类 使用skipgram
#     trainCLF(inFile,model_clf_sg,vec_skipgram)
    
#     trainCLF(inFile,model_clf_CBOW,vec_CBOW)
    
    # 定义clf模型
    bin_clf_sg = model_clf_sg + ".bin"
    
    # 定义clf模型
    bin_clf_CBOW = model_clf_CBOW + ".bin"
    
    # 测试
#     pred(testFile,bin_clf_sg,outFile)

In [ ]:
if __name__ == '__main__':
    main()